In [6]:
import openai
import requests
import os
import json
from concurrent.futures import ThreadPoolExecutor
import random

# API key configuration from .env file in the root directory
api_key = os.getenv("OPENAI_API_KEY")

# Models configuration 
model_config = {
    'chat_model': {"options": ["gpt-3.5-turbo-0125", "gpt-4-turbo-2024-04-09"], "selected": 1},
    'audio_model': {"options": ["tts-1", "tts-1-hd"], "selected": 1},
    'image_model': {"options": ["dall-e-2", "dall-e-3"], "selected": 1}
}


def initialize_openai():
    """Initialize the OpenAI client with the configured API key."""
    return openai.OpenAI(api_key=api_key)


client = initialize_openai()

In [7]:
def select_random_genre():
    """Selects a random genre and a random starting scene from predefined options."""

    genres_data = {
        "genres": [
            {
                "name": "Adventure",
                "writer": "Robert Louis Stevenson",
                "artist": "A Renaissance painter of great renown",
                "initial_scene_description": "Amidst the verdant embrace of a tropical island, you stand at the threshold of an ancient path that twists eastward beneath the towering canopy. Every leaf and vine is painted with the vivid hues that only the stroke of nature, and perhaps a Renaissance master, could conceive. This path, carpeted with fallen blossoms and the detritus of many rains, beckons you deeper into its secrets and away from the sound of the distant surf.",
                "voice": "onyx",
                "time": "6:45 AM"
            },
            {
                "name": "Horror",
                "writer": "Edgar Allan Poe",
                "artist": "a daguerreotype photographer of great renown",
                "initial_scene_description": "The chill of an overcast day in a deserted seaside town grips your bones as you find yourself at the mouth of a narrow alley. This alley, like a forgotten vein of the old town, runs north into the heart of an encroaching fog that seems almost alive. The monochrome hues of the abandoned shops and homes around you suggest the silvered daguerreotypes of a time long past, each window and shadow capturing the essence of a ghost story waiting to be told.",
                "voice": "shimmer",
                "time": "7:00 PM"
            },
            {
                "name": "Science Fiction",
                "writer": "Neal Stephenson",
                "artist": "H. R. Giger",
                "initial_scene_description": "You find yourself standing at the edge of a vast, desolate plaza. The skeletons of towering skyscrapers loom like sentinels around the perimeter, their once-gleaming facades now dulled and pitted by the relentless passage of time. To the east, a monolithic structure's shattered glass creates a mosaic of reflections from the weak, diffused sunlight filtering through the dense overcast sky. A weathered road, cracked and overgrown with resilient flora, offers a path eastward, winding through the heart of this ghostly metropolis.",
                "voice": "onyx",
                "time": "12:00 PM"
            }
        ]
    }

    random_genre = random.choice(genres_data["genres"])

    story_data = {
        'name': random_genre["name"],
        'writer': random_genre["writer"],
        'artist': random_genre["artist"],
        'initial_scene_description': random_genre["initial_scene_description"],
        'time': random_genre["time"],
        'voice': random_genre["voice"],
        'previous_scene': '',
        'player_move': '',
    }

    print(json.dumps(story_data, indent=4))
    return story_data

In [8]:
def generate_prompt(story_data):
    """
    Generates a prompt for the client based on the story_data.
    
    Args:
        story_data (dict): A dictionary containing details about the selected genre, including writer, name, and scene descriptions.
    
    Returns:
        list: A list containing the constructed prompt as a dictionary.
    """
    # Determine the scene description and back option based on the presence of a previous scene
    if story_data['previous_scene'].strip():
        scene_description = f"a previous game scene which read {story_data['previous_scene']} and the player chose to move {story_data['player_move']}."
        back_option = "Additionally, include an option to 'back', allowing the player to go back to the previous location."
    else:
        scene_description = story_data['initial_scene_description']
        back_option = "This scene should not offer the player a 'back' option, as it is the starting location of the game."
    
    foundation_prompt = f"You are {story_data['writer']}, writing a {story_data['name']} text game in your literary style for a twelve-year-old. The scene to write is based on {scene_description} at {story_data['time']}. Do not name the player or give them any tools or weapons. Please return a detailed JSON object with keys for 'story', 'scene_description', and 'player_options'. The JSON object structure must be as follows: "

    data_shape_prompt = f"The scene description is to be used by Dall-E to generate a visualization of the story you are generating from the player's first-person perspective. The scene description should be detailed and vivid. Always feature at least one path for the player to move in. The scene_description MUST be under 500 characters. The player options should include only those directions that are explicitly mentioned in the scene description (e.g., if the scene describes a pathway leading forward, only 'forward' should be an option). {back_option} 'player_options' should be structured as an object with a 'direction' key which should be an array containing the direction (the options here are 'forward', 'left', 'right', 'back' only) and a 'command text' like ['forward', 'Continue down alley'], consisting of three words maximum and a 'transition_text' key. The transition_text should describe the action of moving in that direction as it happens, like 'As you step forward, your shoes echo on the cobblestone, piercing the silence as the fog envelops you like a cold embrace.' Keep these texts to one or two sentences. JSON object structure must be as follows: story, scene_description, player_options. player_options should contain 'directions' which will have all available directions that player can move. each direction should contain the 'direction', 'command_text', and 'transition_text'."

    # Construct the client prompt
    prompt = [{
        "role": "system",
        "content": f"{foundation_prompt} {data_shape_prompt}"
    }]
    
    return prompt


In [9]:
def generate_content(client, prompt):
    """Fetch content from OpenAI based on an initial prompt and return a detailed JSON object."""
    response = client.chat.completions.create(
        model=model_config['chat_model']['options'][model_config['chat_model']['selected']],
        messages=prompt,
        response_format={"type": "json_object"}
    )
    if response.choices[0].finish_reason != 'length':
        content = json.loads(response.choices[0].message.content)
        # Verify all expected keys are in the response
        expected_keys = {'story', 'scene_description', 'player_options'}
        if not expected_keys <= content.keys():
            raise ValueError("Some expected keys are missing in the response")
        # Pretty print the JSON content
        print(json.dumps(content, indent=4))
        return content
    else:
        raise ValueError("Response was cut off due to length limit.")


In [11]:
story_data = select_random_genre()

{
    "name": "Horror",
    "writer": "Edgar Allan Poe",
    "artist": "a daguerreotype photographer of great renown",
    "initial_scene_description": "The chill of an overcast day in a deserted seaside town grips your bones as you find yourself at the mouth of a narrow alley. This alley, like a forgotten vein of the old town, runs north into the heart of an encroaching fog that seems almost alive. The monochrome hues of the abandoned shops and homes around you suggest the silvered daguerreotypes of a time long past, each window and shadow capturing the essence of a ghost story waiting to be told.",
    "time": "7:00 PM",
    "voice": "shimmer",
    "previous_scene": "",
    "player_move": ""
}


In [12]:
def generate_audio(description, voice):
    url = "https://api.openai.com/v1/audio/speech"
    headers = {"Authorization": f"Bearer {api_key}",
               "Content-Type": "application/json"}
    data = {"model": model_config['audio_model']['options']
            [model_config['audio_model']['selected']], "input": description, "voice": voice}
    response = requests.post(url, json=data, headers=headers)
    if response.status_code == 200:
        with open('output_3.mp3', 'wb') as f:
            f.write(response.content)
        return "Audio has been saved as output.mp3"
    return "Failed to synthesize speech"


In [13]:
def generate_image(description, artist):
    url = "https://api.openai.com/v1/images/generations"
    headers = {"Authorization": f"Bearer {api_key}",
               "Content-Type": "application/json"}
    data = {
        "model": model_config['image_model']['options'][model_config['image_model']['selected']],
        "prompt": f"You are {artist} creating this scene: {description} for a text adventure game. The image should be from the eyeline of the player and very true to the description. The painting should include all aspects of the description, and always include any mentioned paths or walkways. All pathways MUST be shown PROMINENTLY. No text should be included in the image.",
        "size": "1024x1024",
        "n": 1,
        "response_format": "url"
    }

    # Making the API call
    response = requests.post(url, json=data, headers=headers)
    if response.status_code == 200:
        image_url = response.json()['data'][0]['url']
        image_response = requests.get(image_url)
        if image_response.status_code == 200:
            image_path = 'scene_image_3.png'
            with open(image_path, 'wb') as f:
                f.write(image_response.content)
            return f"Image has been saved as {image_path}"
        else:
            return "Failed to download the image"
    else:
        return f"Failed to generate image: {response.status_code}, {response.text}"

In [14]:
generated_prompt = generate_prompt(story_data)
print(generated_prompt)

[{'role': 'system', 'content': "You are Edgar Allan Poe, writing a Horror text game in your literary style for a twelve-year-old. The scene to write is based on The chill of an overcast day in a deserted seaside town grips your bones as you find yourself at the mouth of a narrow alley. This alley, like a forgotten vein of the old town, runs north into the heart of an encroaching fog that seems almost alive. The monochrome hues of the abandoned shops and homes around you suggest the silvered daguerreotypes of a time long past, each window and shadow capturing the essence of a ghost story waiting to be told. at 7:00 PM. Do not name the player or give them any tools or weapons. Please return a detailed JSON object with keys for 'story', 'scene_description', and 'player_options'. The JSON object structure must be as follows:  The scene description is to be used by Dall-E to generate a visualization of the story you are generating from the player's first-person perspective. The scene descri

In [15]:
scene = generate_content(client, generated_prompt)

{
    "story": "As the last rays of a cheerless sun retreat behind the clouded horizon, you find yourself standing at the entrance of an obscure, narrow alley in a long-deserted seaside town. The chilly air and the oppressive silence amplify the mysterious allure of the fog that thickens with every passing moment, clawing its way through the empty streets. The desolated buildings lining either side of the alley whisper tales of yesteryears, their muted colors and eerie stillness painting a scene of forgotten sorrow and dread.",
    "scene_description": "A narrow alley stretches from a deserted seaside town into an encroaching, seemingly alive fog. Abandoned shops and homes line the path, their empty windows and shadows under a monochrome sky evoke a ghostly ambiance. No signs of life, only the quiet and a chilling breeze.",
    "player_options": {
        "directions": [
            {
                "direction": "forward",
                "command_text": "Continue down alley",
       

In [16]:
generate_audio(scene['story'], story_data['voice'])

'Audio has been saved as output.mp3'

In [17]:
generate_image(scene['scene_description'], story_data['artist'])

'Image has been saved as scene_image_3.png'

In [18]:
story_data['previous_scene'] = scene['story']
story_data['player_move'] = "forward"
generated_prompt = generate_prompt(story_data)

In [19]:
print(generated_prompt)

[{'role': 'system', 'content': "You are Edgar Allan Poe, writing a Horror text game in your literary style for a twelve-year-old. The scene to write is based on a previous game scene which read As the last rays of a cheerless sun retreat behind the clouded horizon, you find yourself standing at the entrance of an obscure, narrow alley in a long-deserted seaside town. The chilly air and the oppressive silence amplify the mysterious allure of the fog that thickens with every passing moment, clawing its way through the empty streets. The desolated buildings lining either side of the alley whisper tales of yesteryears, their muted colors and eerie stillness painting a scene of forgotten sorrow and dread. and the player chose to move forward. at 7:00 PM. Do not name the player or give them any tools or weapons. Please return a detailed JSON object with keys for 'story', 'scene_description', and 'player_options'. The JSON object structure must be as follows:  The scene description is to be u

In [20]:
scene = generate_content(client, generated_prompt)


{
    "story": "As you march deeper into the shadow-ridden corridor of the alley, an eerie chill seems to seep through your very bones. The flickering of an old gaslight, hanging precariously at a bend further ahead, casts ghostly shadows that dance upon the decrepit walls. The narrow path, bounded by the looming, derelict structures, beckons you silently under the cloak of an ever-thickening fog. Its wisps whisper secrets of forgotten tales and hidden fears. No sound dares break the haunting silence save for your solitary footsteps, which resound with a hollow echo against the ancient cobblestones. Mysterious inscriptions etched onto the walls near the gaslight seem to pulsate faintly with a timeless forewarning, urging caution with each solitary step forward.",
    "scene_description": "A fading gaslight illuminates the dark, narrow alley lined by derelict buildings. Shadows flicker against the walls under a dense, creeping fog. Ancient cobblestones lead up to the gaslight where myst

In [21]:
generate_audio(scene['story'], story_data['voice'])


'Audio has been saved as output.mp3'

In [22]:
generate_image(scene['scene_description'], story_data['artist'])

'Image has been saved as scene_image_3.png'

In [23]:
story_data['previous_scene'] = scene['story']
story_data['player_move'] = "forward"
generated_prompt = generate_prompt(story_data)

In [24]:
print(generated_prompt)

[{'role': 'system', 'content': "You are Edgar Allan Poe, writing a Horror text game in your literary style for a twelve-year-old. The scene to write is based on a previous game scene which read As you march deeper into the shadow-ridden corridor of the alley, an eerie chill seems to seep through your very bones. The flickering of an old gaslight, hanging precariously at a bend further ahead, casts ghostly shadows that dance upon the decrepit walls. The narrow path, bounded by the looming, derelict structures, beckons you silently under the cloak of an ever-thickening fog. Its wisps whisper secrets of forgotten tales and hidden fears. No sound dares break the haunting silence save for your solitary footsteps, which resound with a hollow echo against the ancient cobblestones. Mysterious inscriptions etched onto the walls near the gaslight seem to pulsate faintly with a timeless forewarning, urging caution with each solitary step forward. and the player chose to move forward. at 7:00 PM. 

In [25]:
scene = generate_content(client, generated_prompt)

{
    "story": "As you decide to move forward, the corridor narrows, and despite the fear clawing at your senses, an intense curiosity propels you onward. There is a palpable shift in the air as you approach the foreboding end of the alley. A faint, almost indiscernible light flickers in the distance, grasping at the edges of darkness, casting eerie, elongated shadows that seem almost sentient. The chilling silence is suddenly punctuated by a soft whisper, as though the night itself spoke in hushed, spectral breaths. The ground beneath your feet feels unsteady, and the faint inscriptions, growing ever more vivid, seem to writhe under the dim glow of the distant light.",
    "scene_description": "A narrow, shadow-ridden corridor ends at a flickering light in the distance. The ground feels unsteady, and eerie shadows stretch across the walls. Faint, pulsating inscriptions grow vivid beneath the dim, distant light.",
    "player_options": {
        "directions": [
            {
          

In [26]:
generate_audio(scene['story'], story_data['voice'])

'Audio has been saved as output.mp3'

In [27]:
generate_image(scene['scene_description'], story_data['artist'])

'Image has been saved as scene_image_3.png'